In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

In [2]:
# GOAL: Residual Connections
# - Show gradient flow with/without skip connections
# - Implement simple ResNet block

In [3]:
a = torch.randn((3, 3))
a

tensor([[ 0.5204,  0.6766, -1.6369],
        [-0.1860, -2.1445,  0.9022],
        [ 0.5416,  0.1288,  0.3409]])

In [ ]:
# need a simple and deep network
# 1. passing the input to the deep network
# 2. checking the mean/std/variance of gradient in network stream
# 3. define a simple optimizer (for 50 steps)
# 4. check the mean/std/variance of gradients in the end without residual connections
# 5. check the mean/std/variance of gradients in the end where residual connections used in network
